In [1]:
from Acesso import Login
from Query import Query
from Moeda import Moeda
import os
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

querys={

    'Devolucao':

    """
    
    SELECT *
    FROM netfeira.vw_devolucao
    WHERE [Data de Entrada]=DATEADD(DAY,-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)) 
    AND [Situação do Pedido]<>'CANCELADO'    
    
    """,

    'Vendedor':

    """
    
    SELECT * FROM netfeira.vw_vendedor
    WHERE [Status do Vendedor]='ATIVO'
    
    """,

    'Supervisor':

    """
    
    SELECT * FROM netfeira.vw_supervisor
    
    """,

    'Produto':

    """
   
    SELECT SKU,Produto,Fabricante 
    FROM netfeira.vw_produto

    """,

    'Cliente':

    """
    
    SELECT [ID Cliente],[Razão Social],[Nome Fantasia] 
    FROM netfeira.vw_cliente
    
    """

}

In [2]:
df=sql.CriarTabela(kwargs=querys)

In [10]:
for tabela in df.keys():
    
    display(df[tabela].columns)
    
    pass

Index(['Data de Entrada', 'Motivo', 'Usuário Responsável',
       'Situação do Pedido', 'Tipo de Pedido', 'Tipo de Operação', 'Tabelas',
       'Pedido', 'NFe', 'ID Cliente', 'ID Vendedor', 'SKU', 'Qtde',
       'Unid. VDA', 'Qtde VDA', 'Valor Unitário', 'Total Geral'],
      dtype='object')

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'Status do Vendedor', 'DDD',
       'Telefone'],
      dtype='object')

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup', 'DDD Sup',
       'Telefone Sup', 'ID Gerente', 'Gerente', 'Email Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')

Index(['SKU', 'Produto', 'Fabricante'], dtype='object')

Index(['ID Cliente', 'Razão Social', 'Nome Fantasia'], dtype='object')

In [15]:
df['Vendedor']=df['Vendedor'].merge(df['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'Status do Vendedor', 'DDD',
       'Telefone','ID Sup', 'Supervisor', 'Email Sup', 'DDD Sup',
       'Telefone Sup', 'ID Gerente', 'Gerente', 'Email Gerente', 'DDD Gerente',
       'Telefone Gerente']]

In [18]:
df['Devolucao']=df['Devolucao'].merge(df['Cliente'],on='ID Cliente',how='inner')

In [19]:
df['Devolucao']=df['Devolucao'].merge(df['Produto'],on='SKU',how='inner')

In [20]:
df['Devolucao']=df['Devolucao'].merge(df['Vendedor'],on='ID Vendedor',how='inner')

In [22]:
df['Devolucao'].columns

Index(['Data de Entrada', 'Motivo', 'Usuário Responsável',
       'Situação do Pedido', 'Tipo de Pedido', 'Tipo de Operação', 'Tabelas',
       'Pedido', 'NFe', 'ID Cliente', 'ID Vendedor', 'SKU', 'Qtde',
       'Unid. VDA', 'Qtde VDA', 'Valor Unitário', 'Total Geral',
       'Razão Social', 'Nome Fantasia', 'Produto', 'Fabricante', 'Vendedor',
       'Nome Resumido', 'Equipe', 'E-mail', 'Categoria', 'Data de Cadastro',
       'Status do Vendedor', 'DDD', 'Telefone', 'ID Sup', 'Supervisor',
       'Email Sup', 'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente',
       'Email Gerente', 'DDD Gerente', 'Telefone Gerente'],
      dtype='object')

In [27]:
df['Devolucao']=df['Devolucao'][['Data de Entrada', 'Motivo',
       'Situação do Pedido', 'Tipo de Pedido', 'Tipo de Operação', 'Tabelas',
       'Pedido', 'NFe', 'ID Cliente','Razão Social', 'Nome Fantasia', 'ID Vendedor','Vendedor',
       'Nome Resumido', 'Equipe', 'E-mail', 'Categoria', 'Data de Cadastro',
       'Status do Vendedor', 'DDD', 'Telefone', 'ID Sup', 'Supervisor',
       'Email Sup', 'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente',
       'Email Gerente', 'DDD Gerente', 'Telefone Gerente', 'SKU','Produto', 'Fabricante', 'Qtde',
       'Unid. VDA', 'Qtde VDA', 'Valor Unitário', 'Total Geral']]

In [64]:
colunas={'ID Vendedor':'ID Sup','ID Sup':'ID Gerente','ID Gerente':'ID Gerente'}

nomes={'ID Vendedor':'Nome Resumido','ID Sup':'Supervisor','ID Gerente':'Gerente'}

col_ddd={'ID Vendedor':'DDD','ID Sup':'DDD Sup','ID Gerente':'DDD Gerente'}

col_tel={'ID Vendedor':'Telefone','ID Sup':'Telefone Sup','ID Gerente':'Telefone Gerente'}

for col1,col2 in colunas.items():
    
    codigos=df['Devolucao'][col1].loc[(df['Devolucao']['Categoria']=='CLT')&(~df['Devolucao']['Telefone'].isnull())].unique().tolist()
    
    for c in codigos:
        
        temp_df=pd.DataFrame()
        
        sup=df['Devolucao'][col2].loc[df['Devolucao'][col1]==c].unique().tolist()[-1]
        
        if(col1=='ID Sup'):
            
            nome=str(df['Devolucao'][nomes[col1]].loc[df['Devolucao'][col1]==c].unique().tolist()[-1]).title()
            
            ddd=df['Devolucao'][col_ddd[col1]].loc[df['Devolucao'][col1]==c].unique().tolist()[-1]
            
            telefone=df['Devolucao'][col_tel[col1]].loc[df['Devolucao'][col1]==c].unique().tolist()[-1]
            
            temp_df=df['Devolucao'].loc[df['Devolucao'][col1]==c]
            
            pass
        
        else:
        
            nome=str(df['Devolucao'][nomes[col1]].loc[df['Devolucao'][col1]==c].unique().tolist()[-1]).title()
            
            ddd=df['Devolucao'][col_ddd[col1]].loc[df['Devolucao'][col1]==c].unique().tolist()[-1]
            
            telefone=df['Devolucao'][col_tel[col1]].loc[df['Devolucao'][col1]==c].unique().tolist()[-1]
            
            temp_df=df['Devolucao'].loc[df['Devolucao'][col1]==c]
                        
            pass
                
        display(temp_df)
        
        pass
        
    pass

,Data de Entrada,Motivo,Situação do Pedido,Tipo de Pedido,Tipo de Operação,Tabelas,Pedido,NFe,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,DDD,Telefone,ID Sup,Supervisor,Email Sup,DDD Sup,Telefone Sup,ID Gerente,Gerente,Email Gerente,DDD Gerente,Telefone Gerente,SKU,Produto,Fabricante,Qtde,Unid. VDA,Qtde VDA,Valor Unitário,Total Geral
8,2022-11-30,ERRO DE DIGITAÇÃO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,443334,973134,22184,LEONARDO ROSA MINI PADARIA,PADARIA DO ALEMÃO,RENATOP,RENATO PEREIRA ROCHA,RENATO ROCHA,EQUIPE 6 (CLT),RENATO.ROCHA@DEMARCHISAOPAULO.COM.BR,CLT,2018-03-26,ATIVO,11.0,996870636.0,CARLOSH,CARLOS PASSOS,CARLOS.HENRIQUE@DEMARCHIBRASIL.COM.BR,11,942987560,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,1120,"CONG. BATATA MCCAIN 9MM TRAD 1,5KG (10X1,5KG)",MCCAIN,10.0,UN,10.0,20.8,208.0
9,2022-11-30,ERRO DE DIGITAÇÃO,DEVOLUÇÃO,BONIFICAÇÃO DEMARCHI,BONIFICAÇÃO,TABELA DE PREÇO,443336,973135,22184,LEONARDO ROSA MINI PADARIA,PADARIA DO ALEMÃO,RENATOP,RENATO PEREIRA ROCHA,RENATO ROCHA,EQUIPE 6 (CLT),RENATO.ROCHA@DEMARCHISAOPAULO.COM.BR,CLT,2018-03-26,ATIVO,11.0,996870636.0,CARLOSH,CARLOS PASSOS,CARLOS.HENRIQUE@DEMARCHIBRASIL.COM.BR,11,942987560,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,1120,"CONG. BATATA MCCAIN 9MM TRAD 1,5KG (10X1,5KG)",MCCAIN,1.0,UN,1.0,20.8,20.8


,Data de Entrada,Motivo,Situação do Pedido,Tipo de Pedido,Tipo de Operação,Tabelas,Pedido,NFe,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,DDD,Telefone,ID Sup,Supervisor,Email Sup,DDD Sup,Telefone Sup,ID Gerente,Gerente,Email Gerente,DDD Gerente,Telefone Gerente,SKU,Produto,Fabricante,Qtde,Unid. VDA,Qtde VDA,Valor Unitário,Total Geral
19,2022-11-30,ESTABELECIMENTO FECHADO,DEVOLUÇÃO,BONIFICAÇÃO DEMARCHI,BONIFICAÇÃO,TABELA DE PREÇO,444310,973529,24524,COMERCIAL DE ALIMENTOS RED LTDA,COMERCIAL DE ALIMENT,MARCOSLI,MARCOS VINICIUS DE LIMA E SILVA,MARCOS SILVA,EQUIPE 3 (CLT),MARCOS.LIMA@DEMARCHISAOPAULO.COM.BR,CLT,2021-09-13,ATIVO,11.0,965390196.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,3,POLPINHA DE ABACAXI (100G),DE MARCHI,1.0,KG,1.0,22.21,22.21
20,2022-11-30,FORA DE HORARIO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,444304,973528,24524,COMERCIAL DE ALIMENTOS RED LTDA,COMERCIAL DE ALIMENT,MARCOSLI,MARCOS VINICIUS DE LIMA E SILVA,MARCOS SILVA,EQUIPE 3 (CLT),MARCOS.LIMA@DEMARCHISAOPAULO.COM.BR,CLT,2021-09-13,ATIVO,11.0,965390196.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,3,POLPINHA DE ABACAXI (100G),DE MARCHI,1.0,KG,1.0,22.21,22.21
21,2022-11-30,ESTABELECIMENTO FECHADO,DEVOLUÇÃO,BONIFICAÇÃO DEMARCHI,BONIFICAÇÃO,TABELA DE PREÇO,444310,973529,24524,COMERCIAL DE ALIMENTOS RED LTDA,COMERCIAL DE ALIMENT,MARCOSLI,MARCOS VINICIUS DE LIMA E SILVA,MARCOS SILVA,EQUIPE 3 (CLT),MARCOS.LIMA@DEMARCHISAOPAULO.COM.BR,CLT,2021-09-13,ATIVO,11.0,965390196.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,57,POLPINHA DE CAJU (100G),DE MARCHI,1.0,KG,1.0,23.43,23.43
22,2022-11-30,FORA DE HORARIO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,444304,973528,24524,COMERCIAL DE ALIMENTOS RED LTDA,COMERCIAL DE ALIMENT,MARCOSLI,MARCOS VINICIUS DE LIMA E SILVA,MARCOS SILVA,EQUIPE 3 (CLT),MARCOS.LIMA@DEMARCHISAOPAULO.COM.BR,CLT,2021-09-13,ATIVO,11.0,965390196.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,57,POLPINHA DE CAJU (100G),DE MARCHI,1.0,KG,1.0,23.43,23.43
23,2022-11-30,FORA DE HORARIO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,444304,973528,24524,COMERCIAL DE ALIMENTOS RED LTDA,COMERCIAL DE ALIMENT,MARCOSLI,MARCOS VINICIUS DE LIMA E SILVA,MARCOS SILVA,EQUIPE 3 (CLT),MARCOS.LIMA@DEMARCHISAOPAULO.COM.BR,CLT,2021-09-13,ATIVO,11.0,965390196.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,403,POLPINHA DE ACEROLA COM LARANJA (100G),DE MARCHI,1.0,KG,1.0,25.13,25.13
24,2022-11-30,FORA DE HORARIO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,444304,973528,24524,COMERCIAL DE ALIMENTOS RED LTDA,COMERCIAL DE ALIMENT,MARCOSLI,MARCOS VINICIUS DE LIMA E SILVA,MARCOS SILVA,EQUIPE 3 (CLT),MARCOS.LIMA@DEMARCHISAOPAULO.COM.BR,CLT,2021-09-13,ATIVO,11.0,965390196.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,55,POLPINHA DE CAJA (100G),DE MARCHI,1.0,KG,1.0,27.33,27.33
25,2022-11-30,FORA DE HORARIO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,444304,973528,24524,COMERCIAL DE ALIMENTOS RED LTDA,COMERCIAL DE ALIMENT,MARCOSLI,MARCOS VINICIUS DE LIMA E SILVA,MARCOS SILVA,EQUIPE 3 (CLT),MARCOS.LIMA@DEMARCHISAOPAULO.COM.BR,CLT,2021-09-13,ATIVO,11.0,965390196.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,405,POLPINHA DE FRUTAS VERMELHAS (100G),DE MARCHI,1.0,KG,1.0,51.23,51.23
26,2022-11-30,FORA DE HORARIO,DEVOLUÇÃO

,Data de Entrada,Motivo,Situação do Pedido,Tipo de Pedido,Tipo de Operação,Tabelas,Pedido,NFe,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,DDD,Telefone,ID Sup,Supervisor,Email Sup,DDD Sup,Telefone Sup,ID Gerente,Gerente,Email Gerente,DDD Gerente,Telefone Gerente,SKU,Produto,Fabricante,Qtde,Unid. VDA,Qtde VDA,Valor Unitário,Total Geral
30,2022-11-30,FALTA DE PRODUTO,DEVOLUÇÃO PARCIAL,VENDA DEMARCHI,VENDAS,TABELA DE NEGOCIAÇÃO,444433,973781,23666,SUPERMERCADO PORTO SEGURO TABOAO LTDA,SUPERMERCADO PORTO S,CLAUDIOP,CLAUDIO TADEU PEREIRA,CLAUDIO PEREIRA,EQUIPE 3 (CLT),DEMARCHI.CLAUDIOP@GMAIL.COM,CLT,2017-10-26,ATIVO,11.0,974582160.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,9403,CONG. BATATA MCCAIN BINT 9MM (8X2KG),MCCAIN,8.0,CX,1.0,142.4,142.4


,Data de Entrada,Motivo,Situação do Pedido,Tipo de Pedido,Tipo de Operação,Tabelas,Pedido,NFe,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,DDD,Telefone,ID Sup,Supervisor,Email Sup,DDD Sup,Telefone Sup,ID Gerente,Gerente,Email Gerente,DDD Gerente,Telefone Gerente,SKU,Produto,Fabricante,Qtde,Unid. VDA,Qtde VDA,Valor Unitário,Total Geral
31,2022-11-30,FORA DE TEMPERATURA,DEVOLUÇÃO PARCIAL,VENDA DEMARCHI,VENDAS,TABELA DE NEGOCIAÇÃO,444464,973883,24008,SUPERMERCADO MAMBO LTDA,MAMBO CD,DOUGLASM,DOUGLAS MARTINS TEDESCHI,DOUGLAS TEDESCHI,EQUIPE 4 (VAR),DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,CLT,2018-01-04,ATIVO,11.0,998664048.0,DOUGLASM,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,11,998664048,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,350,IQF BLUEBERRY CONGELADA POTE (450G),DE MARCHI,10.0,CX,1.0,183.1,183.1


,Data de Entrada,Motivo,Situação do Pedido,Tipo de Pedido,Tipo de Operação,Tabelas,Pedido,NFe,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,DDD,Telefone,ID Sup,Supervisor,Email Sup,DDD Sup,Telefone Sup,ID Gerente,Gerente,Email Gerente,DDD Gerente,Telefone Gerente,SKU,Produto,Fabricante,Qtde,Unid. VDA,Qtde VDA,Valor Unitário,Total Geral
8,2022-11-30,ERRO DE DIGITAÇÃO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,443334,973134,22184,LEONARDO ROSA MINI PADARIA,PADARIA DO ALEMÃO,RENATOP,RENATO PEREIRA ROCHA,RENATO ROCHA,EQUIPE 6 (CLT),RENATO.ROCHA@DEMARCHISAOPAULO.COM.BR,CLT,2018-03-26,ATIVO,11.0,996870636.0,CARLOSH,CARLOS PASSOS,CARLOS.HENRIQUE@DEMARCHIBRASIL.COM.BR,11,942987560,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,1120,"CONG. BATATA MCCAIN 9MM TRAD 1,5KG (10X1,5KG)",MCCAIN,10.0,UN,10.0,20.8,208.0
9,2022-11-30,ERRO DE DIGITAÇÃO,DEVOLUÇÃO,BONIFICAÇÃO DEMARCHI,BONIFICAÇÃO,TABELA DE PREÇO,443336,973135,22184,LEONARDO ROSA MINI PADARIA,PADARIA DO ALEMÃO,RENATOP,RENATO PEREIRA ROCHA,RENATO ROCHA,EQUIPE 6 (CLT),RENATO.ROCHA@DEMARCHISAOPAULO.COM.BR,CLT,2018-03-26,ATIVO,11.0,996870636.0,CARLOSH,CARLOS PASSOS,CARLOS.HENRIQUE@DEMARCHIBRASIL.COM.BR,11,942987560,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,1120,"CONG. BATATA MCCAIN 9MM TRAD 1,5KG (10X1,5KG)",MCCAIN,1.0,UN,1.0,20.8,20.8


,Data de Entrada,Motivo,Situação do Pedido,Tipo de Pedido,Tipo de Operação,Tabelas,Pedido,NFe,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,DDD,Telefone,ID Sup,Supervisor,Email Sup,DDD Sup,Telefone Sup,ID Gerente,Gerente,Email Gerente,DDD Gerente,Telefone Gerente,SKU,Produto,Fabricante,Qtde,Unid. VDA,Qtde VDA,Valor Unitário,Total Geral
1,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,594,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",MCCAIN,90.0,UN,90.0,41.71,3753.90
2,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,350,IQF BLUEBERRY CONGELADA POTE (450G),DE MARCHI,10.0,UN,10.0,27.90,279.00
3,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,440,"IQF MORANGO (1,2 KG)",DE MARCHI,10.0,UN,10.0,30.95,309.50
4,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,960,ACAI SPORT DM BALDE 910G MORANGO,DE MARCHI,6.0,UN,6.0,20.10,120.60
5,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,959,ACAI SPORT DM BALDE 910G BANANA,DE MARCHI,6.0,UN,6.0,20.10,120.60
6,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,347,IQF AMORA CONGELADA POTE (450G),DE MARCHI,10.0,UN,10.0,34.64,346.40
7,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,972,IQF FRUTAS VERMELHAS CONGELADA POTE 450G,DE MARCHI,10.0,UN,10.0,28.80,288.00
10,2022-11-30,ERRO DE DIGITAÇÃO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,444033,973487,6586,HM HM SUPERMERCADOS LTDA,EMPORIUM SAO PAULO A,FABIANO,FABIANO DA SILVA REPRESENTACOES COMERCIA,F

,Data de Entrada,Motivo,Situação do Pedido,Tipo de Pedido,Tipo de Operação,Tabelas,Pedido,NFe,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,DDD,Telefone,ID Sup,Supervisor,Email Sup,DDD Sup,Telefone Sup,ID Gerente,Gerente,Email Gerente,DDD Gerente,Telefone Gerente,SKU,Produto,Fabricante,Qtde,Unid. VDA,Qtde VDA,Valor Unitário,Total Geral
31,2022-11-30,FORA DE TEMPERATURA,DEVOLUÇÃO PARCIAL,VENDA DEMARCHI,VENDAS,TABELA DE NEGOCIAÇÃO,444464,973883,24008,SUPERMERCADO MAMBO LTDA,MAMBO CD,DOUGLASM,DOUGLAS MARTINS TEDESCHI,DOUGLAS TEDESCHI,EQUIPE 4 (VAR),DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,CLT,2018-01-04,ATIVO,11.0,998664048.0,DOUGLASM,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,11,998664048,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,350,IQF BLUEBERRY CONGELADA POTE (450G),DE MARCHI,10.0,CX,1.0,183.1,183.1


,Data de Entrada,Motivo,Situação do Pedido,Tipo de Pedido,Tipo de Operação,Tabelas,Pedido,NFe,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,DDD,Telefone,ID Sup,Supervisor,Email Sup,DDD Sup,Telefone Sup,ID Gerente,Gerente,Email Gerente,DDD Gerente,Telefone Gerente,SKU,Produto,Fabricante,Qtde,Unid. VDA,Qtde VDA,Valor Unitário,Total Geral
0,2022-11-30,ERRO DE DIGITAÇÃO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,444312,973283,27515,MARCIA MARIA DIAS LENTI,LANCHONETE DA BOCHA,VALDO,LUIZ CARLOS KAZLAUSKAS VALDO,LUIZ VALDO,EQUIPE 1 (REP),,PJ,2017-10-26,ATIVO,11.0,998369050.0,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,594,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",MCCAIN,42.0,UN,42.0,41.00,1722.00
1,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,594,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",MCCAIN,90.0,UN,90.0,41.71,3753.90
2,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,350,IQF BLUEBERRY CONGELADA POTE (450G),DE MARCHI,10.0,UN,10.0,27.90,279.00
3,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,440,"IQF MORANGO (1,2 KG)",DE MARCHI,10.0,UN,10.0,30.95,309.50
4,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,960,ACAI SPORT DM BALDE 910G MORANGO,DE MARCHI,6.0,UN,6.0,20.10,120.60
5,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,959,ACAI SPORT DM BALDE 910G BANANA,DE MARCHI,6.0,UN,6.0,20.10,120.60
6,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELUIZFARIA2011@HOTMAIL.COM,PJ,2017-10-26,ATIVO,11.0,947583464.0,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,11,942986864,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,11,951283394,347,IQF AMORA CONGELADA POTE (450G),DE MARCHI,10.0,UN,10.0,34.64,346.40
7,2022-11-30,PRODUTO DUPLICADO,DEVOLUÇÃO,VENDA DEMARCHI,VENDAS,PASTORINHO,442888,973702,11483,DIAS PASTORINHO SA COMERCIO E INDUSTRIA,DIAS PASTORINHO SANT,JOSEFARI,JOSE FARIA,JOSE FARIA,EQUIPE 2 (REP),JOSELU